In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("turkish_news_70000.csv",index_col = "id")

In [ ]:
df =df.head(1000)
df.head()

## Veri Hazırlama ve Temizleme

In [ ]:
haber_veriseti = df[["text"]]

In [ ]:
haber_veriseti

## Genel veri temizleme işlemleri 
Tüm kelimeleri küçük hale getirme, stop wordsleri kaldırma, noktalama işaretlerini kaldırma

In [ ]:
# Veri temizleme işlemi için, bir fonksiyon tanımlayalım ve 
# Bu fonksiyonu bütün veri üzerinde uygulayalım
# Öncesinde noktalama işaretleri ve storwords kümelerini tanımlayalım
import re
import string
import nltk
from nltk.corpus import stopwords

nok_isaretleri_kümesi = string.punctuation
# nltk kütüphanesinin etkisiz kelimeler kümesini kullanalım 
etkisiz_kelimeler_kümesi = stopwords.words('turkish') 
# stopwords kümesine biz de istediğimiz kelimeleri ekleyebiliriz
# Örnek:
etkisiz_kelimeler_kümesi.extend(["bir","kadar","sonra"])

def veri_temizleme(metin):
    # Metindeki tüm harfleri küçük duruma getirir.
    metin = metin.lower()
    # Verisetimizdeki yeni satır karekterleri, boşluk karekteriyle değiştirdik.  
    metin = metin.replace("\\n"," ")
    # Kesme işareti ve sonrasındaki karekterlerin kaldırılması
    metin = re.sub("’(\w+)", "", metin) 
    metin = re.sub("'(\w+)", "", metin) 
    metin = re.sub("[“,‘,’,”]", "", metin) 
    # Sayıların Kaldırılması
    metin = re.sub("[0-9]+", "", metin)
    # Noktalama işaretlerinin kaldırılması
    metin = "".join(list(map(lambda x:x if x not in nok_isaretleri_kümesi else " ", metin)))
    # Etkisiz kelimelerin bir kısmının kaldırılması
    metin = " ".join([i for i in metin.split() if i not in etkisiz_kelimeler_kümesi])
    # Metinde tek kalan harfleri de çıkartalım
    metin = " ".join([i for i in metin.split() if len(i) > 1])
    
    return metin

In [ ]:
haber_veriseti["temizlenen_metin"] = haber_veriseti["text"].apply(lambda x:veri_temizleme(x))

#### Ham Metin - Temizlenmiş Metin Örneği

In [ ]:
haber_veriseti.iloc[5].text

In [ ]:
haber_veriseti.iloc[5].temizlenen_metin

In [ ]:
haber_veriseti.head()

In [ ]:
haber_veriseti["temizlenen_metin_token"] = haber_veriseti["temizlenen_metin"].apply(lambda x: x.split())

In [ ]:
haber_veriseti.head()

## LDA Model Eğitimi

In [ ]:
import gensim
import pyLDAvis.gensim

In [ ]:
#Kelime Listesi
tokenlastirilmis_metinler = haber_veriseti["temizlenen_metin_token"]
kelime_listesi=gensim.corpora.Dictionary(tokenlastirilmis_metinler)
#kelime listesi filtreleme
kelime_listesi.filter_extremes(no_below=1,no_above=0.7) #?


In [ ]:
#terimlerin vektörleştirilmesi - Doküman terim matrisinin oluşturulması
dokuman_terim_matrisi = [kelime_listesi.doc2bow(terim) for terim in tokenlastirilmis_metinler]

In [ ]:
#LDA Model
lda_model = gensim.models.ldamodel.LdaModel(corpus = dokuman_terim_matrisi,
                                           id2word= kelime_listesi,
                                           num_topics=15,
                                           passes = 10)






## Çıkarımlar

In [ ]:
#Oluşturulan soyut konular içerisinde en fazla bulunan terimler
konular = lda_model.print_topics(num_words = 7)
for konu in konular:
    print(konu)

# En iyi performansı sağlayan konu sayısını belirleme 
## Tutarlılık (Coherence) Skoru ile en iyi konu sayısını seçme

In [ ]:
pyLDAvis.enable_notebook()
gorsel = pyLDAvis.gensim.prepare(lda_model, dokuman_terim_matrisi, kelime_listesi, mds='mmds')
pyLDAvis.display(gorsel)

In [ ]:
from gensim.models import CoherenceModel
konu_sayisi_aralik_listesi = range(9,30,3)

In [ ]:
tutarlilik_skorlar_listesi=list()
konu_sayisi_listesi= list()

for konu_sayisi in konu_sayisi_aralik_listesi:
    lda_model = gensim.models.ldamodel.LdaModel(corpus = dokuman_terim_matrisi,
                                           id2word= kelime_listesi,
                                           num_topics=konu_sayisi,
                                           passes = 10)
    tutarlilik_model_lda=CoherenceModel(model=lda_model,texts=tokenlastirilmis_metinler,
                                       dictionary=kelime_listesi , coherence= "c_v")
    gecici_tutarlilik_skoru_lda=tutarlilik_model_lda.get_coherence()
    tutarlilik_skorlar_listesi.append(gecici_tutarlilik_skoru_lda)
    konu_sayisi_listesi.append(konu_sayisi)
    
    
    

In [ ]:
import matplotlib.pyplot as plt
plt.plot(konu_sayisi_listesi,tutarlilik_skorlar_listesi,"-"),
plt.xlabel("Konu Sayıları")
plt.ylabel("Tutarlılık Skorları")
plt.show()

In [ ]:
 #Oluşturulan soyut konular içerisinde en fazla bulunan terimler
konular = lda_model.print_topics(num_words = 7)
konular = sorted(konular,key = lambda x : x[0])
for konu in konular:
    print(konu)

In [ ]:
pyLDAvis.enable_notebook()
gorsel = pyLDAvis.gensim.prepare(lda_model, dokuman_terim_matrisi, kelime_listesi, mds='mmds')
pyLDAvis.display(gorsel)